In [ ]:
import numpy as np

# 假设有以下全局变量
num_clients = N  # 客户端数量
num_rounds = R  # 迭代次数
selected_clients_per_round = K  # 每轮选择的客户端数量
learning_rate = lr  # 学习率

# 初始化全局模型参数（这里以numpy数组表示）
global_model_params = np.random.randn(param_shape)

# 伪代码实现FedGSCS算法
for round in range(num_rounds):
    # 1. 客户端梯度计算
    gradients = []
    for client_id in range(num_clients):
        # 在客户端上训练模型并计算梯度（这里省略具体实现）
        local_gradient = train_and_compute_gradient(client_id, global_model_params)
        gradients.append(local_gradient)
    
    # 2. 平均梯度计算
    average_gradient = np.mean(gradients, axis=0)
    
    # 3. 客户端选择
    similarities = []
    for client_gradient in gradients:
        similarity = np.dot(client_gradient, average_gradient) / (np.linalg.norm(client_gradient) * np.linalg.norm(average_gradient))
        similarities.append(similarity)
    
    selected_clients = np.argsort(similarities)[-selected_clients_per_round:]  # 选择相似度最高的K个客户端
    
    # 4. 模型聚合
    new_global_model_params = global_model_params.copy()
    for client_id in selected_clients:
        # 从选定的客户端获取本地模型参数（这里省略具体实现）
        local_model_params = get_local_model_params(client_id)
        # 更新全局模型参数（这里以简单的平均聚合为例）
        new_global_model_params += learning_rate * (local_model_params - global_model_params) / selected_clients_per_round
    
    # 更新全局模型参数
    global_model_params = new_global_model_params

# 输出最终的全局模型参数（或其他评估指标）
print(global_model_params)

# 注意：上述伪代码中省略了具体的训练、计算梯度和获取本地模型参数的函数实现，这些需要根据实际应用场景进行具体实现。

In [2]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """
    计算两个向量之间的余弦相似性
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def fed_gscs(clients_gradients, top_q):
    """
    FedGSCS算法实现
    
    参数:
    clients_gradients: 客户端梯度的列表，每个元素是一个numpy数组
    top_q: 需要选择的客户端数量
    
    返回:
    selected_clients: 被选中的客户端索引列表
    """
    # 计算平均梯度
    average_gradient = np.mean(clients_gradients, axis=0)
    
    # 初始化相似度列表
    similarities = []
    
    # 计算每个客户端梯度与平均梯度的余弦相似度
    for i, gradient in enumerate(clients_gradients):
        similarity = cosine_similarity(gradient, average_gradient)
        similarities.append((i, similarity))
    
    # 根据相似度排序
    similarities.sort(key=lambda x: x[1], reverse=True)
    print(similarities)
    # 选择前top_q个客户端
    selected_clients = [client[0] for client in similarities[:top_q]]
    
    return selected_clients

# 示例数据
# 假设有4个客户端，每个客户端的梯度是一个长度为5的向量
clients_gradients = [
    np.array([1.0, 2.0, 3.0, 4.0, 5.0]),
    np.array([2.0, 3.0, 4.0, 5.0, 6.0]),
    np.array([-1.0, -2.0, -3.0, -4.0, -5.0]),
    np.array([0.5, 1.0, 1.5, 2.0, 2.5])
]

# 选择前2个客户端
top_q = 2
selected_clients = fed_gscs(clients_gradients, top_q)
print("Selected clients:", selected_clients)

[(1, 0.9996002398401119), (0, 0.9973804801107508), (3, 0.9973804801107508), (2, -0.9973804801107508)]
Selected clients: [1, 0]
